# Train a Model to Trade on the Stock Market

![](https://www.xm.com/wp-content/uploads/2017/09/Stock-Charts-1.jpg)

## Efficient Market Hypothesis

Future prices cannot be predicted by analyzing prices from the past. Excess returns cannot be earned in the long run by using investment strategies based on historical share prices or other historical data. Technical analysis techniques will not be able to consistently produce excess returns.

 

## Technical Analysis


In finance, technical analysis is an analysis methodology for forecasting the direction of prices through the study of past market data, primarily price and volume. The efficacy of technical  analysis is disputed by the efficient-market hypothesis which states that stock market prices are essentially unpredictable.

 

## Can a neural net learn Technical Analysis techniques to predict the market and make good trades?

Are we about to get rich?

In [1]:
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries
import random
from indicators import *
from transform import *
import pandas_datareader as web
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import h2o
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import cufflinks as cf
from plots import *
import numpy as np
import operator
import math

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

# Set some defaults for how pandas displays
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Enter These Variables

In [2]:
stockSymbol = "MSFT"
market = "NASDAQ"
invest = 10000
window = 20
start_date = pd.datetime(2013,1,1)
split_validation = pd.datetime(2017,1,1)
split_test = pd.datetime(2017,10,1)

### Let's use a machine learning library called H2O

In [3]:
#h2o.cluster().shutdown()

In [4]:
h2o.init(max_mem_size="12G")
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,10 hours 4 mins
H2O cluster timezone:,America/Toronto
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.8
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_brett_08igme
H2O cluster total nodes:,1
H2O cluster free memory:,10.51 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


### Connect to Morningstar and Download the Historical Data  

In [5]:
df = web.DataReader(stockSymbol, 'morningstar', start=start_date)
df = df.reset_index(drop=False)
df = df.drop('Symbol', axis=1)
df.set_index('Date', inplace=True)
df = df[['Open','High', 'Low', 'Close','Volume']]


### Plot.ly is a Sweet Charting Library

In [6]:
myPlotlyKey = 'conh5EnFad0Z9Lz6mVWr'
py.sign_in(username='bretto777',api_key=myPlotlyKey)

fig = simpleStockChart(df,stockSymbol,split_validation,split_test)
py.iplot(fig, validate=False)

The draw time for this plot will be slow for clients without much RAM.


### Technical Analysis
Add technical analysis indicators used by the pros! Hopefully our model will learn how these indicators can predict tomorrow's price movements.

In [7]:
%%capture
df.reset_index(drop=False, inplace=True, col_level=0)

typical_price(df, high_col = 'High', low_col = 'Low', close_col = 'Close')
money_flow_index(df, vol_col='Volume')
rsi(df, close_col='Close')
ema(df, period=26, column='Close')
ema(df, period=12, column='Close')
macd(df, period_long=26, period_short=12, period_signal=9, column='Close')
on_balance_volume(df, trend_periods=21, close_col='Close', vol_col='Volume')
price_volume_trend(df, trend_periods=21, close_col='Close', vol_col='Volume')
bollinger_bands(df, trend_periods=20, close_col='Close')

In [8]:
fig = technicalAnalysisChart(df)
py.iplot(fig, validate=False)

In [9]:
%%capture
windowTransform(df, window)

In [10]:
df['Close Tomorrow Trend'].replace(0,-1,inplace=True)    

trainingDF = df.loc[df['Date'] <= split_validation]
validationDF = df[df['Date'].isin(pd.date_range(split_validation,split_test))]
testDF = df.loc[df['Date'] > split_test]

trainingDF = trainingDF.dropna(axis=0, how='any')
validationDF = validationDF.dropna(axis=0, how='any')
testDF = testDF.fillna(value=0)

trainingDF.set_index('Date', inplace=True)
validationDF.set_index('Date', inplace=True)
testDF.set_index('Date', inplace=True)

In [11]:
trainingDF.tail(5)

,Open,High,Low,Close,Volume,typical_price,money_flow,money_ratio,money_flow_index,money_flow_positive,money_flow_negative,rsi,ema26,ema12,macd_val,macd_signal_line,obv,obv_ema21,pvt,pvt_ema21,bol_bands_middle,bol_bands_upper,bol_bands_lower,Close Tomorrow,Close Tomorrow Change,Close Tomorrow Trend,Open-1,Open Change-0,Open-1 Trend,Open-2,Open Change-1,Open-2 Trend,Open-3,Open Change-2,Open-3 Trend,Open-4,Open Change-3,Open-4 Trend,Open-5,Open Change-4,Open-5 Trend,Open-6,Open Change-5,Open-6 Trend,Open-7,Open Change-6,Open-7 Trend,Open-8,Open Change-7,Open-8 Trend,Open-9,Open Change-8,Open-9 Trend,Open-10,Open Change-9,Open-10 Trend,Open-11,Open Change-10,Open-11 Trend,Open-12,Open Change-11,Open-12 Trend,Open-13,Open Change-12,Open-13 Trend,Open-14,Open Change-13,Open-14 Trend,Open-15,Open Change-14,Open-15 Trend,Open-16,Open Change-15,Open-16 Trend,Open-17,Open Change-16,Open-17 Trend,Open-18,Open Change-17,Open-18 Trend,Open-19,Open Change-18,Open-19 Trend,High-1,High Change-0,High-1 Trend,High-2,High Change-1,High-2 Trend,High-3,High Change-2,High-3 Trend,High-4,High Change-3,High-4 Trend,High-5,High Change-4,High-5 Trend,High-6,High Change-5,High-6 Trend,High-7,High Change-6,High-7 Trend,High-8,High Change-7,High-8 Trend,High-9,High Change-8,High-9 Trend,High-10,High Change-9,High-10 Trend,High-11,High Change-10,High-11 Trend,High-12,High Change-11,High-12 Trend,High-13,High Change-12,High-13 Trend,High-14,High Change-13,High-14 Trend,High-15,High Change-14,High-15 Trend,High-16,High Change-15,High-16 Trend,High-17,High Change-16,High-17 Trend,High-18,High Change-17,High-18 Trend,High-19,High Change-18,High-19 Trend,Low-1,Low Change-0,Low-1 Trend,Low-2,Low Change-1,Low-2 Trend,Low-3,Low Change-2,Low-3 Trend,Low-4,Low Change-3,Low-4 Trend,Low-5,Low Change-4,Low-5 Trend,Low-6,Low Change-5,Low-6 Trend,Low-7,Low Change-6,Low-7 Trend,Low-8,Low Change-7,Low-8 Trend,Low-9,Low Change-8,Low-9 Trend,Low-10,Low Change-9,Low-10 Trend,Low-11,Low Change-10,Low-11 Trend,Low-12,Low Change-11,Low-12 Trend,Low-13,Low Change-12,Low-13 Trend,Low-14,Low Change-13,Low-14 Trend,Low-15,Low Change-14,Low-15 Trend,Low-16,Low Change-15,Low-16 Trend,Low-17,Low Change-16,Low-17 Trend,Low-18,Low Change-17,Low-18 Trend,Low-19,Low Change-18,Low-19 Trend,Close-1,Close Change-0,Close-1 Trend,Close-2,Close Change-1,Close-2 Trend,Close-3,Close Change-2,Close-3 Trend,Close-4,Close Change-3,Close-4 Trend,Close-5,Close Change-4,Close-5 Trend,Close-6,Close Change-5,Close-6 Trend,Close-7,Close Change-6,Close-7 Trend,Close-8,Close Change-7,Close-8 Trend,Close-9,Close Change-8,Close-9 Trend,Close-10,Close Change-9,Close-10 Trend,Close-11,Close Change-10,Close-11 Trend,Close-12,Close Change-11,Close-12 Trend,Close-13,Close Change-12,Close-13 Trend,Close-14,Close Change-13,Close-14 Trend,Close-15,Close Change-14,Close-15 Trend,Close-16,Close Change-15,Close-16 Trend,Close-17,Close Change-16,Close-17 Trend,Close-18,Close Change-17,...,BBand-Lower-11 Trend,BBand-Middle-11,BBand-Middle Change-10,BBand-Middle-11 Trend,BBand-Upper-12,BBand-Upper Change-11,BBand-Upper-12 Trend,BBand-Lower-12,BBand-Lower Change-11,BBand-Lower-12 Trend,BBand-Middle-12,BBand-Middle Change-11,BBand-Middle-12 Trend,BBand-Upper-13,BBand-Upper Change-12,BBand-Upper-13 Trend,BBand-Lower-13,BBand-Lower Change-12,BBand-Lower-13 Trend,BBand-Middle-13,BBand-Middle Change-12,BBand-Middle-13 Trend,BBand-Upper-14,BBand-Upper Change-13,BBand-Upper-14 Trend,BBand-Lower-14,BBand-Lower Change-13,BBand-Lower-14 Trend,BBand-Middle-14,BBand-Middle Change-13,BBand-Middle-14 Trend,BBand-Upper-15,BBand-Upper Change-14,BBand-Upper-15 Trend,BBand-Lower-15,BBand-Lower Change-14,BBand-Lower-15 Trend,BBand-Middle-15,BBand-Middle Change-14,BBand-Middle-15 Trend,BBand-Upper-16,BBand-Upper Change-15,BBand-Upper-16 Trend,BBand-Lower-16,BBand-Lower Change-15,BBand-Lower-16 Trend,BBand-Middle-16,BBand-Middle Change-15,BBand-Middle-16 Trend,BBand-Upper-17,BBand-Upper Change-16,BBand-Upper-17 Trend,BBand-Lower-17,BBand-Lower 

In [12]:
train = h2o.H2OFrame(trainingDF, destination_frame="train.hex")
valid = h2o.H2OFrame(validationDF, destination_frame="valid.hex")
test = h2o.H2OFrame(testDF, destination_frame="test.hex")

train['Close Tomorrow Trend'] = train['Close Tomorrow Trend'].asfactor()
train = train.drop('Close Tomorrow', axis=1)
train = train.drop('Close Tomorrow Change', axis=1)
valid['Close Tomorrow Trend'] = valid['Close Tomorrow Trend'].asfactor()
valid = valid.drop('Close Tomorrow', axis=1)
valid = valid.drop('Close Tomorrow Change', axis=1)
test['Close Tomorrow Trend'] = test['Close Tomorrow Trend'].asfactor()
test = test.drop('Close Tomorrow', axis=1)
test = test.drop('Close Tomorrow Change', axis=1)

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = train.columns
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator as H2OPCA

pca_decomp = H2OPCA(k=100, transform="NONE", pca_method="Power")
pca_decomp.train(x=x, training_frame=train)
trainReduced = pca_decomp.predict(train)
trainReduced = trainReduced.cbind(train['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=valid)
validReduced = pca_decomp.predict(valid)
validReduced = validReduced.cbind(valid['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=test)
testReduced = pca_decomp.predict(test)
testReduced = testReduced.cbind(test['Close Tomorrow Trend'])

pca Model Build progress: |███████████████████████████████████████████████

### Train the Deep Neural Net
Grid search for a good model

In [ ]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

hidden_opt = [[800],
              [1000],
              [2000],
              [500,500],
              [25,25,25],
              [50,75,75,50],
              [75,100,100,75],
              [50,300,300,50],
              [1000,1000],
              [50,50,50]]
l1_opt = [0.001,0.0001,0.00001]
l2_opt = [0.001,0.0001]
rate_opt= [0.01,0.001,0.0001]
activation_opt=["Rectifier","TanhWithDropout"]
epochs_opt=[3500,2500]

hyper_parameters = {"hidden":hidden_opt, 
                    "l1":l1_opt, 
                    "l2":l2_opt,
                    "rate":rate_opt, 
                    "activation":activation_opt,
                    "epochs":epochs_opt}

search_criteria = {"strategy":"RandomDiscrete", "max_models":5, "seed":123456}



model_grid = H2OGridSearch(H2ODeepLearningEstimator(),
                           hyper_params=hyper_parameters,
                           search_criteria=search_criteria,
                           grid_id='test8')
model_grid.train(x=x, 
                 y=y,                                   
                 fast_mode=True,                   
                 stopping_rounds=10, 
                 stopping_tolerance=0,
                 stopping_metric='AUC',
                 adaptive_rate=True,                
                 training_frame=trainReduced, 
                 validation_frame=validReduced,
                 score_interval=10)

In [ ]:
model_gridperf1 = model_grid.get_grid(sort_by='AUC', decreasing=False)
bestModel = model_gridperf1[0]          
#h2o.save_model(model=bestModel, path="/home/brett/")


In [ ]:
#bestModel = h2o.load_model(path="/home/brett/ticker/h2ogbm.h2o")

In [ ]:
importances = bestModel.varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

In [ ]:
cm = bestModel.confusion_matrix()
cm = cm.as_data_frame()
confusionMatrix = FF.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

In [ ]:
pBestModel = pd.concat([bestModel.predict(test_data=test).as_data_frame(use_pandas=True).reset_index(drop=True), testDF.reset_index(drop=False)], axis =1)


In [ ]:
prices = df.loc[df['Date'] > split_test]
prices = prices.loc[:,['Date','Close','Close Tomorrow','Close Tomorrow Trend']]
profit = pBestModel.loc[:,['predict']]
profit = pd.concat([prices.reset_index(drop=True), profit.reset_index(drop=True)], axis =1)
profit['predict'] = profit['predict'].astype(np.float64)

profit['profit'] = invest

profit['Best Possible'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['Close Tomorrow Trend'] > 0 else 0, axis=1)

profit['Randomly Invest'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if random.randint(1,1000) > random.randint(1,1000) else 0, axis=1)
profit['profit'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['predict'] > 0 else 0, axis=1)


print "Times " + stockSymbol + " actually went up    : " + str(profit[profit['Close Tomorrow Trend']>0].count()['Close Tomorrow Trend'])
print "Times " + stockSymbol + " predicted to go up  : " + str(profit[profit['predict']>0].count()['predict'])
print "Profit                        : $" + str(profit.profit.sum())
print "Random investment             : $" + str(profit['Randomly Invest'].sum())
print "Best possible outcome         : $" + str(profit['Best Possible'].sum())

profit

